In [1]:
from __future__ import print_function, division
 
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
import torch.backends.cudnn as cudnn
import random
import math
import cv2
import numpy as np
%matplotlib inline

import skimage.transform
from torch.autograd import Variable
import torch.nn.functional as F
from PIL import Image
from matplotlib.pyplot import imshow
from torchvision.utils import save_image
from visualize import visualize, reverse_normalize
from imagenet_labels import label2idx, idx2label
import numbers
import warnings
from collections.abc import Sequence
from typing import Tuple, List, Optional

from torch import Tensor

try:
    import accimage
except ImportError:
    accimage = None
from torchvision.transforms.functional import InterpolationMode, _interpolation_modes_from_int

# CamExtractor

In [2]:
class CamExtractor():
    """
        Extracts cam features from the model
    """
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer

    def forward_pass_on_convolutions(self, x):
        """
            Does a forward pass on convolutions, hooks the function at given layer
        """
        conv_output = None
        for module_pos, module in self.model.features._modules.items():
            x = module(x)  # Forward
            if int(module_pos) == self.target_layer:
                conv_output = x  # Save the convolution output on that layer
        return conv_output, x

    def forward_pass(self, x):
        """
            Does a full forward pass on the model
        """
        # Forward pass on the convolutions
        conv_output, x = self.forward_pass_on_convolutions(x)
        x = x.view(x.size(0), -1)  # Flatten
        # Forward pass on the classifier
        x = self.model.classifier(x)
        return conv_output, x
def preprocess_image(pil_im, resize_im=True):
    """
        Processes image for CNNs

    Args:
        PIL_img (PIL_img): PIL Image or numpy array to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    #ensure or transform incoming image to PIL image
    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print("could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im = pil_im.resize((224, 224), Image.ANTIALIAS)

    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var


def FIMF_score_cam(img):

    model_path = './model_vgg16_traffic.pth'
    pretrained_model=torch.load(model_path,map_location='cpu')
    pretrained_model.eval()
    target_layer = 28
    extractor = CamExtractor(pretrained_model, target_layer)
    conv_output, model_output = extractor.forward_pass(img)
    prob = F.softmax(model_output, dim=1)
    p, target_class = torch.max(prob, dim=1)
    target = conv_output[0]
    # Create empty numpy array for cam
    cam = np.zeros(target.shape[1:], dtype=np.float32)

    saliency_map1 = torch.unsqueeze(torch.unsqueeze(target[0, :, :],0),0)
    target_numpy = target.cpu().detach().numpy()

    # Multiply each weight with its conv output and then, sum
    for i in range(len(target)):

        if target[i, :, :].max() == target[i, :, :].min():
            continue

        #template = (target_numpy[i,:,:] - target_numpy[i,:,:].min()) / (target_numpy[i,:,:].max() - target_numpy[i,:,:].min())
        template = np.where(target_numpy[i, :, :]>0,1,0)
        activations_cuda = [target_numpy[k, :, :]*template for k in range(len(target))]
        activations_cuda = np.array(activations_cuda)[None]

        activations_cuda = torch.from_numpy(activations_cuda).float()

        mm = pretrained_model.avgpool(pretrained_model.features[30](pretrained_model.features[29](activations_cuda)))
        mm = mm.view(mm.size(0), -1)

        # Get the target score
        w = F.softmax(pretrained_model.classifier(mm),dim=1)[0][target_class]
        cam += w.data.cpu().numpy() * target[i, :, :].data.cpu().numpy()
    cam = np.maximum(cam, 0)
    cam = (cam - np.min(cam)) / (np.max(cam) - np.min(cam))  # Normalize between 0-1
    cam = np.uint8(cam * 255)  # Scale between 0-255 to visualize
    cam = np.uint8(Image.fromarray(cam).resize((img.shape[2],img.shape[3]), Image.ANTIALIAS))/255
    return(cam)

# GuidedErasing

In [4]:
class GuidedErasing(torch.nn.Module):

    def __init__(self, p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False):
        super().__init__()
        if not isinstance(value, (numbers.Number, str, tuple, list)):
            raise TypeError("Argument value should be either a number or str or a sequence")
        if isinstance(value, str) and value != "random":
            raise ValueError("If value is str, it should be 'random'")
        if not isinstance(scale, (tuple, list)):
            raise TypeError("Scale should be a sequence")
        if not isinstance(ratio, (tuple, list)):
            raise TypeError("Ratio should be a sequence")
        if (scale[0] > scale[1]) or (ratio[0] > ratio[1]):
            warnings.warn("Scale and ratio should be of kind (min, max)")
        if scale[0] < 0 or scale[1] > 1:
            raise ValueError("Scale should be between 0 and 1")
        if p < 0 or p > 1:
            raise ValueError("GuidedErasing probability should be between 0 and 1")

        self.p = p
        self.scale = scale
        self.ratio = ratio
        self.value = value
        self.inplace = inplace

    @staticmethod
    def get_params(img):
        """Get parameters for ``erase`` for a GuidedErasing.

        Args:
            img (Tensor): Tensor image to be erased.
            scale (sequence): range of proportion of erased area against input image.
            ratio (sequence): range of aspect ratio of erased area.
            value (list, optional): erasing value. If None, it is interpreted as "random"
                (erasing each pixel with random values). If ``len(value)`` is 1, it is interpreted as a number,
                i.e. ``value[0]``.

        Returns:
            tuple: params (i, j, h, w, v) to be passed to ``erase`` for GuidedErasing.
        """
        img_c, img_h, img_w = img.shape[-3], img.shape[-2], img.shape[-1]
        area = img_h * img_w

#         log_ratio = torch.log(torch.tensor(ratio))
        for _ in range(10):
            
            img_cam = img.unsqueeze(0)
            heatmap = FIMF_score_cam(img_cam)

            resdivide = np.split(heatmap,4,axis=0)
            resdivide = np.split(np.array(resdivide),4,axis=2)

            sum_box = []
            for i in range(4):
                for j in range(4):
                    sum_box.append(np.array(resdivide)[j][i].sum())

            sum_box -= np.min(sum_box)

            sum_box = sum_box / np.max(sum_box)
            select_erasing = np.where(sum_box>=random.random())
            return select_erasing

        return 0, 0, img_h, img_w, img

    def forward(self, img):
        """
        Args:
            img (Tensor): Tensor image to be erased.

        Returns:
            img (Tensor): Erased Tensor image.
        """
        
        if torch.rand(1) < self.p:

            # cast self.value to script acceptable type
            if isinstance(self.value, (int, float)):
                value = [self.value, ]
            elif isinstance(self.value, str):
                value = None
            elif isinstance(self.value, tuple):
                value = list(self.value)
            else:
                value = self.value

            if value is not None and not (len(value) in (1, img.shape[-3])):
                raise ValueError(
                    "If value is a sequence, it should have either a single value or "
                    "{} (number of input channels)".format(img.shape[-3])
                )

            select_erasing_result = self.get_params(img)
            for i in np.array(select_erasing_result)[0]:
                img[:,56*int(np.floor(i/4)) : 56*int(np.floor(i/4))+56, 56*int(i%4) : 56*int(i%4)+56] = 0
                
        return img

# Training model

In [5]:
use_gpu = torch.cuda.is_available()
data_transforms = {
    'train': transforms.Compose([

        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        GuidedErasing(p=0.5)        

    ]),
    'val': transforms.Compose([
        transforms.Resize([224,224]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
 
data_dir = './data1'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=64,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
 
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
cudnn.benchmark = True

def train_model(model, criterion, optimizer, scheduler, num_epochs=20):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
 
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
 
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train(True)
                scheduler.step()
            else:
                model.train(False)
 
            running_loss = 0.0
            running_corrects = 0
            for data in dataloaders[phase]:
                inputs, labels = data
                optimizer.zero_grad()
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)
 
                if phase == 'train':

                    loss.backward()
                    optimizer.step()
 
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = float(running_corrects) / dataset_sizes[phase]
 
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
 
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
 
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model

#from Vgg16Net import *
if __name__ ==  '__main__':
 
    model = models.vgg16(pretrained=True)
    num_input = model.classifier[6].in_features
    feature_model = list(model.classifier.children())
    feature_model.pop()
    feature_model.append(nn.Linear(num_input, 6))
    model.classifier = nn.Sequential(*feature_model)
    criterion = nn.CrossEntropyLoss()
 
    fc_params = list(map(id, model.classifier[6].parameters()))
    base_params = filter(lambda p: id(p) not in fc_params, model.parameters())
    params = [{"params": base_params, "lr":0.0001},
              {"params": model.classifier[6].parameters(), "lr":0.001},]
    optimizer_ft = torch.optim.SGD(params, momentum=0.9)
 
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=5, gamma=0.1)
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15)
    torch.save(model,"./guide_erasing_model.pth")    

Epoch 0/14
----------


/home/zhengmen/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
1212cpu
torch.Size([3, 224, 224])
1212cpu
12fff1
torch.Size([3, 224, 224])
cpu2232
12fff1
cpu2232
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
cpu5151
cpu5151
cpu5151
121
858
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
4
5
6
8
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
4
5
6
8
9
10
11
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
4
6
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
9
10
11
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
121
858
10
1212cpu
torch.Size([3,

121
858
5
6
8
9
10
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
3
7
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
6
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
10
11
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
4
5
6
7
8
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
6
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
3
4
7
8
9
10
11
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
6
9
10
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
2
3
4
5
7
9
11
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
10
121

12fff1
cpu2232
cpu5151
121
858
0
1
2
3
4
6
7
8
9
10
11
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
2
5
6
9
10
11
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
10
11
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
4
5
8
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
2
3
4
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
2
3
4
5
6
7
9
10
11
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
3
12
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
7
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
12
13
1

...

12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
2
3
4
6
7
9
10
12
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
6
7
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
6
7
12
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
5
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
121
858
1
4
5
8
9
10
11
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
cpu5151
121
858
5
6
8
9
10
11
12
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
3
5
6
9
10
11
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
11
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
3
5


5
6
8
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
4
5
6
7
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
121
858
5
6
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
2
4
5
6
7
8
9
10
11
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
4
5
6
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
2
3
4
5
6
7
8
9
10
11
12
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
4
5
6
7
8
9
10
11
12
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu223

torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
4
5
6
8
9
10
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
12
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
5
9
12
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
9
10
11
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
6
9
10
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
7
8
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
4
5
6
8
9
10
11
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
9
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
8
9
10
12
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu515

12fff1
cpu2232
cpu5151
121
858
9
10
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
6
7
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
5
6
8
9
10
11
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
2
5
6
7
8
9
10
11
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
13
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
4
5
6
8
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
1
2
3
4
5
6
7
8
9
10
11
12
13
14
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
5
6
8
9
10
11
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cpu5151
121
858
4
5
6
7
8
9
10
11
12
13
14
15
1212cpu
torch.Size([3, 224, 224])
12fff1
cpu2232
cp

RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 205520896 bytes. Error code 12 (Cannot allocate memory)